<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Walker_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Minimal interactive Walker Thermostat (Walker, Hays & Kasting, 1981)
# Requirements: numpy, matplotlib, ipywidgets (Colab: you may need to run once:
# %pip install ipywidgets==8 ; from google.colab import output; output.enable_custom_widget_manager(); Runtime->Restart)
# ================================================================
# & F = normalized volcanic outgassing (F_out / W0).
#   F = 1 → present-day reference; F > 1 = stronger volcanism, more CO₂;
#   F < 1 = weaker volcanism, less CO₂.
#
# & β (beta) = exponent for CO₂–weathering sensitivity.
#   Small β (~0.2) = weak feedback; Large β (~0.6) = strong feedback,
#   climate more self-stabilizing.
#
# & T* = temperature scaling constant (K).
#   Small T* (~10 K) → strong thermal feedback, quick response.
#   Large T* (~20 K) → weaker thermal feedback, sluggish response.
#   EXAMPLES:
#   High β + low T* = very stable climate (fast reaction to both CO₂ and warming).
#   Low β + high T* = sluggish climate (weak reaction, bigger swings in CO₂ and temperature).
#
# & S2x = climate sensitivity for CO₂ doubling (K).
#   Typical values 2–4.5 K. Defines how much warming occurs if CO₂ doubles.
#
# & Λ (Lambda) = climate sensitivity factor (K per ln(CO₂ ratio)).
#   Λ = S2x / ln(2). Higher Λ = stronger climate response to CO₂ changes.
#
#   Explanation:
#   S2x is the warming knob. It tells us how much the planet heats up if CO₂ doubles.
#   Λ is general: warming for any proportional CO₂ change.
#   β and T* are the cooling knobs. They control how strongly chemical weathering
#   reacts (to CO₂ and to temperature) to pull CO₂ back down.

#
# & P/P0 = relative atmospheric CO₂ concentration.
#   1 = present-day CO₂; >1 = more CO₂; <1 = less CO₂.
#   Typical range for exercises: 0.7–1.5.
#
# & ΔT = surface temperature anomaly (K).
#   ΔT > 0 = warmer climate; ΔT < 0 = cooler climate.
#   Classroom values often between –3 and +3 K.
# ================================================================

import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# ASCII equations implemented:
# W/W0 = (P/P0)^beta * exp(ΔT / Tstar)
# ΔT = Lambda * ln(P / P0),  Lambda = S2x / ln(2)
# At steady state: F = W/W0  ⇒  (P/P0) = F^(1/(beta + Lambda/Tstar)),  ΔT = Lambda * ln(P/P0)

LN2 = math.log(2.0)

def walker_equilibrium(F=1.0, beta=0.3, Tstar=13.7, S2x=3.0, F_min=0.5, F_max=1.5):
    # Compute Lambda and exponent; enforce physical condition
    Lambda = S2x / LN2
    expo = beta + (Lambda / Tstar)
    if expo <= 0:
        raise ValueError("Physical condition violated: beta + Lambda/Tstar must be > 0.")

    # Equilibrium relations
    P_ratio = F ** (1.0 / expo)         # (P/P0)
    dT = Lambda * math.log(P_ratio)     # ΔT

    # Prepare curves over the chosen F-range
    Fs = np.linspace(F_min, F_max, 300)
    P_eq = Fs ** (1.0 / expo)
    dT_eq = Lambda * np.log(P_eq)

    # Brief numeric summary and one-line interpretation
    print(f"Parameters  | beta={beta:.3f}, Tstar={Tstar:.3f} K, S2x={S2x:.3f} K, Lambda={Lambda:.3f} K")
    print(f"Equilibrium | F={F:.3f} ⇒ P/P0={P_ratio:.3f}, ΔT={dT:.3f} K")
    if F > 1.0:
        print("Interpretation: outgassing > reference → CO2↑ → ΔT > 0 (warmer).")
    elif F < 1.0:
        print("Interpretation: outgassing < reference → CO2↓ → ΔT < 0 (cooler).")
    else:
        print("Interpretation: F=1 is the reference state (P/P0=1, ΔT=0).")

    # Plot 1: ΔT (K) vs F with vertical guide at F=1 and current-state marker
    plt.figure(figsize=(6,4))
    plt.plot(Fs, dT_eq)
    plt.axvline(1.0, linestyle='--')
    plt.scatter([F], [dT], s=30)
    plt.xlabel('F = F_out / W0')
    plt.ylabel('ΔT eq (K)')
    plt.title('Equilibrium ΔT (K) vs F')
    plt.grid(True, linestyle=':')
    plt.show()

    # Plot 2: P/P0 vs F with vertical guide at F=1 and current-state marker
    plt.figure(figsize=(6,4))
    plt.plot(Fs, P_eq)
    plt.axvline(1.0, linestyle='--')
    plt.scatter([F], [P_ratio], s=30)
    plt.xlabel('F = F_out / W0')
    plt.ylabel('P/P0 (relative pCO2)')
    plt.title('Equilibrium P/P0 vs F')
    plt.grid(True, linestyle=':')
    plt.show()

    return P_ratio, dT

# Interactive sliders (no seaborn; matplotlib only; self-contained)
interact(
    walker_equilibrium,
    F=FloatSlider(value=1.0, min=0.5, max=1.5, step=0.01, description='F'),
    beta=FloatSlider(value=0.3, min=0.2, max=0.6, step=0.01, description='beta'),
    Tstar=FloatSlider(value=13.7, min=8.0, max=25.0, step=0.1, description='T* (K)'),
    S2x=FloatSlider(value=3.0, min=1.5, max=6.0, step=0.1, description='S2x (K)'),
    F_min=FloatSlider(value=0.5, min=0.1, max=0.9, step=0.01, description='F min'),
    F_max=FloatSlider(value=1.5, min=1.1, max=3.0, step=0.01, description='F max'),
);

interactive(children=(FloatSlider(value=1.0, description='F', max=1.5, min=0.5, step=0.01), FloatSlider(value=…